In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 12.6 MB/s 


In [ ]:
import numpy as np
import torch
import pathlib
import logging
import shutil
import time
import random
import h5py
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
import torchvision
from tensorboardX import SummaryWriter

In [ ]:
args = {
    'seed': 42,
    'resolution': 320,
    'challenge': 'singlecoil',
    'data_path': pathlib.Path('/content/drive/MyDrive/Dataset'),
    'sample_rate': 1.,
    'accelerations': [4, 8],
    'center_fractions': [0.08, 0.04],

    'num_pools': 4,
    'drop_prob': 0.0,
    'num_chans': 32,
    'batch_size': 16,
    'num_epochs': 2,
    'lr': 0.001,
    'lr_step_size': 40,
    'lr_gamma': 0.1,
    'weight_decay': 0.,
    'report_interval': 100,
    'data_parallel': False,
    'device': 'cuda',
    'exp_dir': pathlib.Path('/content/drive/MyDrive/checkpoints'),
    'resume': True,
    'checkpoint': pathlib.Path('/content/drive/MyDrive/checkpoints/model.pt')
}

In [ ]:
def to_tensor(data):
  if np.iscomplexobj(data):
      data = np.stack((data.real, data.imag), axis=-1)
  return torch.from_numpy(data)


def apply_mask(data, mask_func, seed=None):
  shape = np.array(data.shape)
  shape[:-3] = 1
  mask = mask_func(shape, seed)
  return torch.where(mask == 0, torch.Tensor([0]), data), mask


def fft2(data):
  assert data.size(-1) == 2
  data = ifftshift(data, dim=(-3, -2))
  data = torch.fft.fft(data, dim=2, norm='backward')
  data = fftshift(data, dim=(-3, -2))
  return data


def ifft2(data):
  assert data.size(-1) == 2
  data = ifftshift(data, dim=(-3, -2))
  data = torch.fft.ifft(data, dim=2, norm='backward')
  data = fftshift(data, dim=(-3, -2))
  return data


def complex_abs(data):
  assert data.size(-1) == 2
  return (data ** 2).sum(dim=-1).sqrt()


def root_sum_of_squares(data, dim=0):
  return torch.sqrt((data ** 2).sum(dim))


def center_crop(data, shape):
  assert 0 < shape[0] <= data.shape[-2]
  assert 0 < shape[1] <= data.shape[-1]
  w_from = (data.shape[-2] - shape[0]) // 2
  h_from = (data.shape[-1] - shape[1]) // 2
  w_to = w_from + shape[0]
  h_to = h_from + shape[1]
  return data[..., w_from:w_to, h_from:h_to]


def complex_center_crop(data, shape):
  assert 0 < shape[0] <= data.shape[-3]
  assert 0 < shape[1] <= data.shape[-2]
  w_from = (data.shape[-3] - shape[0]) // 2
  h_from = (data.shape[-2] - shape[1]) // 2
  w_to = w_from + shape[0]
  h_to = h_from + shape[1]
  return data[..., w_from:w_to, h_from:h_to, :]


def normalize(data, mean, stddev, eps=0.):
  return (data - mean) / (stddev + eps)


def normalize_instance(data, eps=0.):
  mean = data.mean()
  std = data.std()
  return normalize(data, mean, std, eps), mean, std


# Helper functions

def roll(x, shift, dim):
  if isinstance(shift, (tuple, list)):
      assert len(shift) == len(dim)
      for s, d in zip(shift, dim):
          x = roll(x, s, d)
      return x
  shift = shift % x.size(dim)
  if shift == 0:
      return x
  left = x.narrow(dim, 0, x.size(dim) - shift)
  right = x.narrow(dim, x.size(dim) - shift, shift)
  return torch.cat((right, left), dim=dim)


def fftshift(x, dim=None):
  if dim is None:
      dim = tuple(range(x.dim()))
      shift = [dim // 2 for dim in x.shape]
  elif isinstance(dim, int):
      shift = x.shape[dim] // 2
  else:
      shift = [x.shape[i] // 2 for i in dim]
  return roll(x, shift, dim)


def ifftshift(x, dim=None):
  if dim is None:
      dim = tuple(range(x.dim()))
      shift = [(dim + 1) // 2 for dim in x.shape]
  elif isinstance(dim, int):
      shift = (x.shape[dim] + 1) // 2
  else:
      shift = [(x.shape[i] + 1) // 2 for i in dim]
  return roll(x, shift, dim)

In [ ]:
class SliceData(Dataset):

  def __init__(self, root, transform, challenge, sample_rate=1):
    if challenge not in ('singlecoil', 'multicoil'):
        raise ValueError('challenge should be either "singlecoil" or "multicoil"')

    self.transform = transform
    self.recons_key = 'reconstruction_esc' if challenge == 'singlecoil' \
        else 'reconstruction_rss'

    self.examples = []
    files = list(pathlib.Path(root).iterdir())
    if sample_rate < 1:
        random.shuffle(files)
        num_files = round(len(files) * sample_rate)
        files = files[:num_files]
    for fname in sorted(files):
      try:
        kspace = h5py.File(fname, 'r')['kspace']
        num_slices = kspace.shape[0]
        self.examples += [(fname, slice) for slice in range(num_slices)]
      except:
        continue

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, i):
    fname, slice = self.examples[i]
    with h5py.File(fname, 'r') as data:
        kspace = data['kspace'][slice]
        target = data[self.recons_key][slice] if self.recons_key in data else None
        return self.transform(kspace, target, data.attrs, fname.name, slice)

In [ ]:
class MaskFunc:

  def __init__(self, center_fractions, accelerations):
    if len(center_fractions) != len(accelerations):
        raise ValueError('Number of center fractions should match number of accelerations')

    self.center_fractions = center_fractions
    self.accelerations = accelerations
    self.rng = np.random.RandomState()

  def __call__(self, shape, seed=None):
    if len(shape) < 3:
        raise ValueError('Shape should have 3 or more dimensions')

    self.rng.seed(seed)
    num_cols = shape[-2]

    choice = self.rng.randint(0, len(self.accelerations))
    center_fraction = self.center_fractions[choice]
    acceleration = self.accelerations[choice]

    # Create the mask
    num_low_freqs = int(round(num_cols * center_fraction))
    prob = (num_cols / acceleration - num_low_freqs) / (num_cols - num_low_freqs)
    mask = self.rng.uniform(size=num_cols) < prob
    pad = (num_cols - num_low_freqs + 1) // 2
    mask[pad:pad + num_low_freqs] = True

    # Reshape the mask
    mask_shape = [1 for _ in shape]
    mask_shape[-2] = num_cols
    mask = torch.from_numpy(mask.reshape(*mask_shape).astype(np.float32))

    return mask

In [ ]:
class ConvBlock(nn.Module):

  def __init__(self, in_chans, out_chans, drop_prob):
      super().__init__()

      self.in_chans = in_chans
      self.out_chans = out_chans
      self.drop_prob = drop_prob

      self.layers = nn.Sequential(
          nn.Conv2d(in_chans, out_chans, kernel_size=3, padding=1),
          nn.InstanceNorm2d(out_chans),
          nn.ReLU(),
          nn.Dropout2d(drop_prob),
          nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1),
          nn.InstanceNorm2d(out_chans),
          nn.ReLU(),
          nn.Dropout2d(drop_prob)
      )

  def forward(self, input):
      return self.layers(input)

  def __repr__(self):
      return f'ConvBlock(in_chans={self.in_chans}, out_chans={self.out_chans}, ' \
          f'drop_prob={self.drop_prob})'


class UnetModel(nn.Module):

  def __init__(self, in_chans, out_chans, chans, num_pool_layers, drop_prob):
      super().__init__()

      self.in_chans = in_chans
      self.out_chans = out_chans
      self.chans = chans
      self.num_pool_layers = num_pool_layers
      self.drop_prob = drop_prob

      self.down_sample_layers = nn.ModuleList([ConvBlock(in_chans, chans, drop_prob)])
      ch = chans
      for i in range(num_pool_layers - 1):
          self.down_sample_layers += [ConvBlock(ch, ch * 2, drop_prob)]
          ch *= 2
      self.conv = ConvBlock(ch, ch, drop_prob)

      self.up_sample_layers = nn.ModuleList()
      for i in range(num_pool_layers - 1):
          self.up_sample_layers += [ConvBlock(ch * 2, ch // 2, drop_prob)]
          ch //= 2
      self.up_sample_layers += [ConvBlock(ch * 2, ch, drop_prob)]
      self.conv2 = nn.Sequential(
          nn.Conv2d(ch, ch // 2, kernel_size=1),
          nn.Conv2d(ch // 2, out_chans, kernel_size=1),
          nn.Conv2d(out_chans, out_chans, kernel_size=1),
      )

  def forward(self, input):
      stack = []
      output = input
      # Apply down-sampling layers
      for layer in self.down_sample_layers:
          output = layer(output)
          stack.append(output)
          output = F.max_pool2d(output, kernel_size=2)

      output = self.conv(output)

      # Apply up-sampling layers
      for layer in self.up_sample_layers:
          output = F.interpolate(output, scale_factor=2, mode='bilinear', align_corners=False)
          output = torch.cat([output, stack.pop()], dim=1)
          output = layer(output)
      return self.conv2(output)

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
class DataTransform:

  def __init__(self, mask_func, resolution, which_challenge, use_seed=True):
    if which_challenge not in ('singlecoil', 'multicoil'):
        raise ValueError(f'Challenge should either be "singlecoil" or "multicoil"')
    self.mask_func = mask_func
    self.resolution = resolution
    self.which_challenge = which_challenge
    self.use_seed = use_seed

  def __call__(self, kspace, target, attrs, fname, slice):
    kspace = to_tensor(kspace)
    # Apply mask
    seed = None if not self.use_seed else tuple(map(ord, fname))
    masked_kspace, mask = apply_mask(kspace, self.mask_func, seed)
    # Inverse Fourier Transform to get zero filled solution
    image = ifft2(masked_kspace)
    # Crop input image
    image = complex_center_crop(image, (self.resolution, self.resolution))
    # Absolute value
    # image = complex_abs(image)
    image = image.abs()
    # Apply Root-Sum-of-Squares if multicoil data
    if self.which_challenge == 'multicoil':
        image = root_sum_of_squares(image)
    # Normalize input
    image, mean, std = normalize_instance(image, eps=1e-11)
    image = torch.clamp(image, min=-6, max=6)

    target = to_tensor(target)
    # Normalize target
    target = normalize(target, mean, std, eps=1e-11)
    target = target.clamp(-6, 6)
    return image, target, mean, std, attrs['norm'].astype(np.float32)

In [ ]:
def create_datasets(args):
  train_mask = MaskFunc(args['center_fractions'], args['accelerations'])
  dev_mask = MaskFunc(args['center_fractions'], args['accelerations'])

  train_data = SliceData(
      root=args['data_path'],
      transform=DataTransform(train_mask, args['resolution'], args['challenge']),
      sample_rate=args['sample_rate'],
      challenge=args['challenge']
  )
  dev_data = SliceData(
      root=args['data_path'],
      transform=DataTransform(dev_mask, args['resolution'], args['challenge'], use_seed=True),
      sample_rate=args['sample_rate'],
      challenge=args['challenge'],
  )
  return dev_data, train_data

In [ ]:
def create_data_loaders(args):
  dev_data, train_data = create_datasets(args)
  display_data = [dev_data[i] for i in range(0, len(dev_data), len(dev_data) // 16)]

  train_loader = DataLoader(
      dataset=train_data,
      batch_size=args['batch_size'],
      shuffle=True,
      num_workers=8,
      pin_memory=True,
  )
  dev_loader = DataLoader(
      dataset=dev_data,
      batch_size=args['batch_size'],
      num_workers=8,
      pin_memory=True,
  )
  display_loader = DataLoader(
      dataset=display_data,
      batch_size=16,
      num_workers=8,
      pin_memory=True,
  )
  return train_loader, dev_loader, display_loader

In [ ]:
def train_epoch(args, epoch, model, data_loader, optimizer, writer):
  model.train()
  avg_loss = 0.
  start_epoch = start_iter = time.perf_counter()
  global_step = epoch * len(data_loader)
  for iter, data in enumerate(data_loader):
      input, target, mean, std, norm = data
      input = input.unsqueeze(1).to(args['device'])
      target = target.to(args['device'])
      input_new = (input[:, :, :, :, 0] + input[:, :, :, :, 1]) / 2
      output = model(input_new).squeeze(1)
      loss = F.l1_loss(output, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      avg_loss = 0.99 * avg_loss + 0.01 * loss.item() if iter > 0 else loss.item()
      writer.add_scalar('TrainLoss', loss.item(), global_step + iter)

      num_epochs = args['num_epochs']
      if iter % args['report_interval'] == 0:
          logging.info(
              f'Epoch = [{epoch:3d}/{num_epochs:3d}] '
              f'Iter = [{iter:4d}/{len(data_loader):4d}] '
              f'Loss = {loss.item():.4g} Avg Loss = {avg_loss:.4g} '
              f'Time = {time.perf_counter() - start_iter:.4f}s',
          )
      start_iter = time.perf_counter()
  return avg_loss, time.perf_counter() - start_epoch

In [ ]:
def evaluate(args, epoch, model, data_loader, writer):
  model.eval()
  losses = []
  start = time.perf_counter()
  with torch.no_grad():
      for iter, data in enumerate(data_loader):
          input, target, mean, std, norm = data
          input = input.unsqueeze(1).to(args['device'])
          target = target.to(args['device'])
          input_new = (input[:, :, :, :, 0] + input[:, :, :, :, 1]) / 2
          output = model(input_new).squeeze(1)

          mean = mean.unsqueeze(1).unsqueeze(2).to(args['device'])
          std = std.unsqueeze(1).unsqueeze(2).to(args['device'])
          target = target * std + mean
          output = output * std + mean

          norm = norm.unsqueeze(1).unsqueeze(2).to(args['device'])
          loss = F.mse_loss(output / norm, target / norm, size_average=False)
          losses.append(loss.item())
      writer.add_scalar('Dev_Loss', np.mean(losses), epoch)
  return np.mean(losses), time.perf_counter() - start

In [ ]:
def visualize(args, epoch, model, data_loader, writer):
  def save_image(image, tag):
    image -= image.min()
    image /= image.max()
    grid = torchvision.utils.make_grid(image, nrow=4, pad_value=1)
    writer.add_image(tag, grid, epoch)

  model.eval()
  with torch.no_grad():
    for iter, data in enumerate(data_loader):
      input, target, mean, std, norm = data
      input = input.unsqueeze(1).to(args['device'])
      target = target.unsqueeze(1).to(args['device'])
      input_new = (input[:, :, :, :, 0] + input[:, :, :, :, 1]) / 2
      output = model(input_new)
      save_image(target, 'Target')
      save_image(output, 'Reconstruction')
      save_image(torch.abs(target - output), 'Error')
      break

In [ ]:
def save_model(args, exp_dir, epoch, model, optimizer, best_dev_loss, is_new_best):
  torch.save(
      {
          'epoch': epoch,
          'args': args,
          'model': model.state_dict(),
          'optimizer': optimizer.state_dict(),
          'best_dev_loss': best_dev_loss,
          'exp_dir': exp_dir
      },
      f=exp_dir / 'model.pt'
  )
  if is_new_best:
      shutil.copyfile(exp_dir / 'model.pt', exp_dir / 'best_model.pt')

In [ ]:
def build_model(args):
  model = UnetModel(
      in_chans=1,
      out_chans=1,
      chans=args['num_chans'],
      num_pool_layers=args['num_pools'],
      drop_prob=args['drop_prob']
  ).to(args['device'])
  return model

In [ ]:
def load_model(checkpoint_file):
  checkpoint = torch.load(checkpoint_file)
  args = checkpoint['args']
  model = build_model(args)
  if args['data_parallel']:
      model = torch.nn.DataParallel(model)
  model.load_state_dict(checkpoint['model'])

  optimizer = build_optim(args, model.parameters())
  optimizer.load_state_dict(checkpoint['optimizer'])
  return checkpoint, model, optimizer

In [ ]:
def build_optim(args, params):
  optimizer = torch.optim.RMSprop(params, args['lr'], weight_decay=args['weight_decay'])
  return optimizer

In [ ]:
def main(args):
    args['exp_dir'].mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(log_dir=args['exp_dir'] / 'summary')

    if args['resume']:
        checkpoint, model, optimizer = load_model(args['checkpoint'])
        args = checkpoint['args']
        best_dev_loss = checkpoint['best_dev_loss']
        start_epoch = checkpoint['epoch']
        del checkpoint
    else:
        model = build_model(args)
        if args['data_parallel']:
            model = torch.nn.DataParallel(model)
        optimizer = build_optim(args, model.parameters())
        best_dev_loss = 1e9
        start_epoch = 0
    logging.info(args)
    logging.info(model)

    train_loader, dev_loader, display_loader = create_data_loaders(args)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args['lr_step_size'], args['lr_gamma'])

    for epoch in range(start_epoch, args['num_epochs']):
        scheduler.step(epoch)
        train_loss, train_time = train_epoch(args, epoch, model, train_loader, optimizer, writer)
        dev_loss, dev_time = evaluate(args, epoch, model, dev_loader, writer)
        visualize(args, epoch, model, display_loader, writer)

        is_new_best = dev_loss < best_dev_loss
        best_dev_loss = min(best_dev_loss, dev_loss)
        save_model(args, args['exp_dir'], epoch, model, optimizer, best_dev_loss, is_new_best)
        num_epochs = args['num_epochs']
        logging.info(
            f'Epoch = [{epoch:4d}/{num_epochs:4d}] TrainLoss = {train_loss:.4g} '
            f'DevLoss = {dev_loss:.4g} TrainTime = {train_time:.4f}s DevTime = {dev_time:.4f}s',
        )
    writer.close()

In [ ]:
random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])

In [ ]:
args['exp_dir'].mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir=args['exp_dir'] / 'summary')

if args['resume']:
    checkpoint, model, optimizer = load_model(args['checkpoint'])
    args = checkpoint['args']
    best_dev_loss = checkpoint['best_dev_loss']
    start_epoch = checkpoint['epoch']
    del checkpoint
else:
    model = build_model(args)
    if args['data_parallel']:
        model = torch.nn.DataParallel(model)
    optimizer = build_optim(args, model.parameters())
    best_dev_loss = 1e9
    start_epoch = 0
logging.info(args)
logging.info(model)

INFO:root:{'seed': 42, 'resolution': 320, 'challenge': 'singlecoil', 'data_path': PosixPath('/content/drive/MyDrive/Dataset'), 'sample_rate': 1.0, 'accelerations': [4, 8], 'center_fractions': [0.08, 0.04], 'num_pools': 4, 'drop_prob': 0.0, 'num_chans': 32, 'batch_size': 16, 'num_epochs': 2, 'lr': 0.001, 'lr_step_size': 40, 'lr_gamma': 0.1, 'weight_decay': 0.0, 'report_interval': 100, 'data_parallel': False, 'device': 'cuda', 'exp_dir': PosixPath('/content/drive/MyDrive/checkpoints'), 'resume': False, 'checkpoint': ''}
INFO:root:UnetModel(
  (down_sample_layers): ModuleList(
    (0): ConvBlock(in_chans=1, out_chans=32, drop_prob=0.0)
    (1): ConvBlock(in_chans=32, out_chans=64, drop_prob=0.0)
    (2): ConvBlock(in_chans=64, out_chans=128, drop_prob=0.0)
    (3): ConvBlock(in_chans=128, out_chans=256, drop_prob=0.0)
  )
  (conv): ConvBlock(in_chans=256, out_chans=256, drop_prob=0.0)
  (up_sample_layers): ModuleList(
    (0): ConvBlock(in_chans=512, out_chans=128, drop_prob=0.0)
    (1):

In [ ]:
train_loader, dev_loader, display_loader = create_data_loaders(args)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args['lr_step_size'], args['lr_gamma'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(start_epoch, args['num_epochs']):
    scheduler.step(epoch)
    train_loss, train_time = train_epoch(args, epoch, model, train_loader, optimizer, writer)
    dev_loss, dev_time = evaluate(args, epoch, model, dev_loader, writer)
    visualize(args, epoch, model, display_loader, writer)

    is_new_best = dev_loss < best_dev_loss
    best_dev_loss = min(best_dev_loss, dev_loss)
    save_model(args, args['exp_dir'], epoch, model, optimizer, best_dev_loss, is_new_best)
    num_epochs = args['num_epochs']
    logging.info(
        f'Epoch = [{epoch:4d}/{num_epochs:4d}] TrainLoss = {train_loss:.4g} '
        f'DevLoss = {dev_loss:.4g} TrainTime = {train_time:.4f}s DevTime = {dev_time:.4f}s',
    )
writer.close()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

In [ ]:
# main(args)

In [ ]:
!nvidia-smi

Tue Nov 30 06:27:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    85W / 149W |   6430MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------